<a href="https://colab.research.google.com/github/JColeman1550/BASEBALL.github.io/blob/main/Post_Game_Reports_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(shiny)
library(dplyr)
library(DT)
library(ggplot2)

Red_Sox_2023$pitch_type <- factor(Red_Sox_2023$pitch_type, levels = c("FF", "SI", "FC", "CS", "CU", "KC", "SL", "CH", "FS", "EP", "ST", 'PO'))
Red_Sox_2023$pitch_name <- factor(Red_Sox_2023$pitch_name, levels = c("4-Seam Fastball", "Sinker", "Cutter", "Curveball", "Knuckle Curve", "Slider", "Changeup", "Split-Finger", "Eephus", "Sweeper", "Pitch Out"))
Red_Sox_2023$stand <- factor(Red_Sox_2023$stand, levels = c("R", "L"))

# Create count column
Red_Sox_2023$count <- paste0(Red_Sox_2023$balls, "-", Red_Sox_2023$strikes)

# Create subsets for behind, ahead, and even counts
behind_counts <- unique(Red_Sox_2023$count[Red_Sox_2023$balls > Red_Sox_2023$strikes])
ahead_counts <- unique(Red_Sox_2023$count[Red_Sox_2023$balls < Red_Sox_2023$strikes])
even_counts <- unique(Red_Sox_2023$count[Red_Sox_2023$balls == Red_Sox_2023$strikes])

# Calculate the percentage of pitches thrown in each zone
in_zone <- Red_Sox_2023[Red_Sox_2023$zone %in% 1:9, ]

ui <- fluidPage(
  theme = shinytheme("flatly"),
  navbarPage("Boston Red Sox",
             tabPanel("Pitchers",
                      column(10, offset = 1,
                             hr(),
                             fluidRow(
                               column(2, selectInput(inputId = "PitcherInput", label = "Select Pitcher", choices = sort(unique(Red_Sox_2023$player_name)))),
                               column(2, selectInput(inputId = "GameInput", label = "Select Game", choices = "")),
                               column(2,
                                      checkboxGroupInput(inputId = "StandInput",
                                                         label = "Select Batter Hand",
                                                         choices = sort(unique(Red_Sox_2023$stand)),
                                                         selected = c("R", "L"))
                               ),
                               column(6,
                                      tagList(
                                        checkboxGroupInput(inputId = "CountInput",
                                                           label = "Select Count",
                                                           choices = list(
                                                             "0-0" = "0-0", "0-1" = "0-1", "0-2" = "0-2",
                                                             "1-0" = "1-0", "1-1" = "1-1", "1-2" = "1-2",
                                                             "2-0" = "2-0", "2-1" = "2-1", "2-2" = "2-2", "3-2" = "3-2"
                                                           ),
                                                           selected = c("0-0", "0-1", "0-2", "1-0", "1-1", "1-2", "2-0", "2-1", "2-2", "3-2"),
                                                           inline = TRUE),
                                        tags$script('
                                                    $(document).ready(function(){
                                                      $("#select-all").on("click", function(){
                                                        $("#CountInput input[type=checkbox]").prop("checked", true).change();
                                                        $("#CountInput input[value=ahead]").prop("checked", true).change();
                                                        $("#CountInput input[value=behind]").prop("checked", true).change();
                                                        $("#CountInput input[value=even]").prop("checked", true).change();
                                                      });
                                                      $("#deselect-all").on("click", function(){
                                                        $("#CountInput input[type=checkbox]").prop("checked", false).change();
                                                        $("#CountInput input[value=ahead]").prop("checked", false).change();
                                                        $("#CountInput input[value=behind]").prop("checked", false).change();
                                                        $("#CountInput input[value=even]").prop("checked", false).change();
                                                      });
                                                    });
                                                    ')
                                      ),
                                      actionLink("select-all", "Select All"), br(),
                                      actionLink("deselect-all", "Deselect All")
                               )
                             ),
                             hr(style="border-color: black;"),
                             wellPanel(style = "background: white; border-color:black; border-width:2px",
                                       fluidRow(
                                         column(2, img(src = "https://upload.wikimedia.org/wikipedia/en/thumb/6/6d/RedSoxPrimary_HangingSocks.svg/1200px-RedSoxPrimary_HangingSocks.svg.png", height = 150, width = 150), align = "center"),
                                         column(4, h2(strong(textOutput("selected_pitcher"))), hr(style="border-color: black;"), style = "padding-right:0px;"),
                                         column(6, h2("Post-Game Report"), hr(style="border-color: black;"), h2(textOutput("selected_game")), align = "right", style = "padding-left:0px;")),
                                       hr(style="border-color: black;"),
                                       fluidRow(
                                         column(10, offset = 1, h3(strong(" ")), dataTableOutput("pitcher_summary_table"), align = "center")
                                       ),
                                       br(), br(), br(),
                                       fluidRow(
                                         column(4, plotOutput("pitch_movement_plot"), align = "center"),
                                         column(4, plotOutput("pitch_release_plot"), align = "center"),
                                         column(4, plotOutput("pitch_location_plot"), align = "center")
                                       ),
                                       br(), br(), br(), br()
                             ),
                             br(),
                             p(em("If the contents of this page appear distorted, please decrease your web browser zoom to 80% or 90%."), align = "center")
                      )
             ),
             tabPanel("Batters",
                      column(10, offset = 1,
                             hr(),
                             fluidRow(
                               column(2, selectInput(inputId = "DistInput", label = "Select Pitcher", choices = sort(unique(BOS_Hitters_2023$player_name)))),
                               column(2, selectInput(inputId = "GamedateInput", label = "Select Game", choices = sort(unique(BOS_Hitters_2023$game_date)))),
                               column(2,
                                      checkboxGroupInput(inputId = "ThrowInput",
                                                         label = "Select Batter Hand",
                                                         choices = sort(unique(BOS_Hitters_2023$stand)),
                                                         selected = c("R", "L"))
                               ),
                               column(6,
                                      tagList(
                                        checkboxGroupInput(inputId = "CountInput",
                                                           label = "Select Count",
                                                           choices = list(
                                                             "0-0" = "0-0", "0-1" = "0-1", "0-2" = "0-2",
                                                             "1-0" = "1-0", "1-1" = "1-1", "1-2" = "1-2",
                                                             "2-0" = "2-0", "2-1" = "2-1", "2-2" = "2-2", "3-2" = "3-2"
                                                           ),
                                                           selected = c("0-0", "0-1", "0-2", "1-0", "1-1", "1-2", "2-0", "2-1", "2-2", "3-2"),
                                                           inline = TRUE),
                                        tags$script('
                                                    $(document).ready(function(){
                                                      $("#select-all").on("click", function(){
                                                        $("#CountInput input[type=checkbox]").prop("checked", true).change();
                                                        $("#CountInput input[value=ahead]").prop("checked", true).change();
                                                        $("#CountInput input[value=behind]").prop("checked", true).change();
                                                        $("#CountInput input[value=even]").prop("checked", true).change();
                                                      });
                                                      $("#deselect-all").on("click", function(){
                                                        $("#CountInput input[type=checkbox]").prop("checked", false).change();
                                                        $("#CountInput input[value=ahead]").prop("checked", false).change();
                                                        $("#CountInput input[value=behind]").prop("checked", false).change();
                                                        $("#CountInput input[value=even]").prop("checked", false).change();
                                                      });
                                                    });
                                                    ')
                                      ),
                                      actionLink("select-all", "Select All"), br(),
                                      actionLink("deselect-all", "Deselect All")
                               )
                             ),
                             hr(style="border-color: black;"),
                             wellPanel(style = "background:white; border-color:black; border-width:2px",
                                       fluidRow(
                                         column(2, img(src = "https://upload.wikimedia.org/wikipedia/en/thumb/6/6d/RedSoxPrimary_HangingSocks.svg/1200px-RedSoxPrimary_HangingSocks.svg.png", height = 150, width = 150), align = "center"),
                                         column(4, h2 (strong(textOutput("selected_batter"))), hr(style="border-color: black;"), style = "padding-right:0px;"),
                                         column(6, h2("Post-Game Report"), hr(style="border-color: black;"), h2(textOutput("selected_game_1")), align = "right", style = "padding-left:0px;")),
                                       hr(style="border-color: black;"),
                                       fluidRow(
                                         column(10, offset = 1, h3(strong(" ")), dataTableOutput("pitch_summary_table"), align = "center")
                                       ),
                                       br(), br(), br(),
                                       fluidRow(
                                         column(4, plotOutput("velo_distribution_chart"), align = "center"),
                                       ),
                                       br(), br(), br(), br()
                             ),
                             br(),
                             p(em("If the contents of this page appear distorted, please decrease your web browser zoom to 80% or 90%."), align = "center")

                      ))

  )

)




server <- function(input, output, session) {

  # Pitcher Tab

  observeEvent(input$PitcherInput, {
    if (is.null(input$PitcherInput))
      return()

    updateSelectInput(session, inputId = "GameInput", label = "Select Game",
                      choices = sort(unique(Red_Sox_2023$game_date[Red_Sox_2023$player_name == input$PitcherInput])))
  })

  output$selected_pitcher <- renderText({
    if (is.null(input$PitcherInput))
      return("Choose pitcher")

    input$PitcherInput
  })

  output$selected_game <- renderText({
    input$GameInput
  })

  filtered_pitcher_data <- reactive({
    if (is.null(input$StandInput))
      return(NULL)

    filtered_data <- Red_Sox_2023[Red_Sox_2023$stand == input$StandInput, ]
    return(filtered_data)
  })

  output$pitcher_summary_table <- renderDataTable({
    if (is.null(input$PitcherInput) || is.null(input$GameInput))
      return()

    selected_counts <- input$CountInput

    table <- Red_Sox_2023 %>%
      filter(player_name == input$PitcherInput, stand == input$StandInput, game_date == input$GameInput, count %in% selected_counts) %>%
      group_by('Pitch' = pitch_name) %>%
      summarize('No.' = n(),
                'Max Velo (MPH)' = round(max(release_speed, na.rm = TRUE),1),
                'Avg. Velo (MPH)' = round(mean(release_speed, na.rm = TRUE),1),
                'Avg. Spin (RPM)' = round(mean(release_spin_rate, na.rm = TRUE),0),
                'BU' = round(mean(release_speed - (release_spin_rate / 1000), na.rm = TRUE), 1),
                'V.Break (in.)' = round(mean(pfx_z * 12, na.rm = TRUE),2),
                'H.Break (in.)' = round(mean(pfx_x * -12, na.rm = TRUE),2),
                'Zone %' = round(sum(zone %in% c(1:9))/n(),1)*100,
                'Strike %' = round(sum(type %in% c("S", "X"))/n(),2)*100,
                'Whiff %' = round(sum(description %in% c("swinging_strike", "swinging_strike_blocked"))/
                                    sum(description %in% c("swinging_strike", "foul", "bunt_foul_tip", "foul_bunt",
                                                           "foul_tip", "hit_into_play", "hit_into_play_no_out",
                                                           "hit_into_play_score", "missed_bunt", "swinging_strike_blocked")),2)*100)
    datatable(table, options = list(dom = 't', columnDefs = list(list(targets = 0, visible = FALSE)))) %>%
      formatStyle(c(1,2), `border-left` = "solid 1px") %>% formatStyle(c(2,5,7), `border-right` = "solid 1px")
  })

  output$pitch_movement_plot <- renderPlot({
    if (is.null(input$PitcherInput) || is.null(input$GameInput))
      return()

    selected_counts <- input$CountInput

    dataFilter <- Red_Sox_2023 %>%
      filter(player_name == input$PitcherInput, stand == input$StandInput, game_date == input$GameInput,
             count %in% selected_counts)

    ggplot(data = dataFilter, aes(x = pfx_x*-1, y = pfx_z, color = pitch_type)) +
      labs(x = "Horizontal Movement (ft.)", y = "Vertical Movement (ft.)", color = " ", title = "Pitch Movement") +
      xlim(-2.5, 2.5) + ylim(-2.5, 2.5) +
      geom_segment(aes(x = 0, y = -2.5, xend = 0, yend = 2.5), size = 2, color = "grey55") +
      geom_segment(aes(x = -2.5, y = 0, xend = 2.5, yend = 0), size = 2, color = "grey55") +
      geom_point(size = 3, na.rm = TRUE) +
      theme_bw() + theme(plot.title = element_text(size = 16, face = "bold", hjust = 0.5)) +
      theme(legend.position = "bottom", legend.text = element_text(size = 12), axis.title = element_text(size = 14))
  }, width = 400, height = 350)

  output$pitch_release_plot <- renderPlot({
    if (is.null(input$PitcherInput) || is.null(input$GameInput))
      return()

    selected_counts <- input$CountInput

    dataFilter <- reactive({
      Red_Sox_2023 %>%
        filter(player_name == input$PitcherInput, stand == input$StandInput, game_date == input$GameInput,
               count %in% selected_counts)
    })
    ggplot(data = dataFilter(), aes(x = release_pos_x, y = release_pos_z, color = pitch_type)) +
      labs(x = "Horizontal Release Point", y = "Vertical Release Point", color = " ", title = "Release") +
      xlim(4, -4) + ylim(2, 7) +
      geom_segment(aes(x = 0, y = -25, xend = 0, yend = 25), size = 2, color = "grey55") +
      geom_segment(aes(x = -25, y = 0, xend = 25, yend = 0), size = 2, color = "grey55") +
      geom_point(size = 3, na.rm = TRUE) +
      theme_bw() + theme(plot.title = element_text(size = 16, face = "bold", hjust = 0.5)) +
      theme(legend.position = "bottom", legend.text = element_text(size = 12), axis.title = element_text(size = 14))
  }, width = 400, height = 350)

  output$pitch_location_plot <- renderPlot({
    if (is.null(input$PitcherInput) || is.null(input$GameInput))
      return()

    selected_counts <- input$CountInput

    dataFilter <- Red_Sox_2023 %>%
      filter(player_name == input$PitcherInput, stand == input$StandInput, game_date == input$GameInput,
             count %in% selected_counts)

    ggplot(data = dataFilter, aes(x = plate_x*-1, y = plate_z, color = pitch_type)) +
      xlim(-3,3) + ylim(0,5) + labs(color = "", title = "Pitch Location") +
      geom_rect(aes(xmin = -0.83, xmax = 0.83, ymin = 1.5, ymax = 3.5), alpha = 0, size = 1, color = "black") +
      geom_segment(aes(x = -0.708, y = 0.15, xend = 0.708, yend = 0.15), size = 1, color = "black") +
      geom_segment(aes(x = -0.708, y = 0.3, xend = -0.708, yend = 0.15), size = 1, color = "black") +
      geom_segment(aes(x = -0.708, y = 0.3, xend = 0, yend = 0.5), size = 1, color = "black") +
      geom_segment(aes(x = 0, y = 0.5, xend = 0.708, yend = 0.3), size = 1, color = "black") +
      geom_segment(aes(x = 0.708, y = 0.3, xend = 0.708, yend = 0.15), size = 1, color = "black") +
      geom_point(size = 3, na.rm = TRUE) +
      theme_bw() + theme(plot.title = element_text(size = 16, face = "bold", hjust = 0.5)) +
      theme(legend.position = "bottom", legend.text = element_text(size = 12), axis.title = element_blank())
  }, width = 400, height = 350)






  # Batter Tab



  output$selected_batter <- renderText({
    if (is.null(input$PitcherInput))
      return()

    input$BatterInput
  })

  output$selected_game_1 <- renderText({
    input$GamedateInput
  })

  filtered_batter_data <- reactive({
    if (is.null(input$ThrowInput))
      return(NULL)

    filtered_data <- BOS_Hitters_2023[BOS_Hitters_2023$p_throws == input$ThrowInput, ]
    return(filtered_data)
  })

  output$batter_summary_table <- renderDataTable({
    if (is.null(input$BatterInput) || is.null(input$GamedateInput))
      return()

    selected_counts <- input$CountInput

    table_1 <- BOS_Hitters_2023 %>%
      filter(player_name == input$BatterInput, game_date == input$GamedateInput, count %in% selected_counts) %>%
      group_by('Pitch' = pitch_name) %>%
      summarize('No.' = n(),
                'Avg. EV' = round(mean(launch_speed, na.rm = TRUE),1),
                'Avg. LA' = round(mean(launch_angle, na.rm = TRUE),1),
                'Max EV' = round(max(launch_speed, na.rm = TRUE),1),
                'Swing %' = round(sum(type %in% c("S"))/n(),1)*100,
                'Whiff %' = round(sum(description %in% c("swinging_strike", "swinging_strike_blocked"))/
                                    sum(description %in% c("swinging_strike", "foul", "bunt_foul_tip", "foul_bunt",
                                                           "foul_tip", "hit_into_play", "hit_into_play_no_out",
                                                           "hit_into_play_score", "missed_bunt", "swinging_strike_blocked")),1)*100)
    datatable(table_1, options = list(dom = 't', columnDefs = list(list(targets = 0, visible = FALSE)))) %>%
      formatStyle(c(1,2), `border-left` = "solid 1px") %>% formatStyle(c(2,5,7), `border-right` = "solid 1px")
  })

  output$spray_chart <- renderPlot({
    if (is.null(input$BatterInput) || is.null(input$GamedateInput))
      return()

    selected_counts <- input$CountInput

    dataFilter <- BOS_Hitters_2023 %>%
      filter(player_name == input$BatterInput, game_date == input$GamedateInput,
             count %in% selected_counts)

    ggplot(data = dataFilter, aes(x = hc_x, y = -hc_y,
                                  text = paste('Date: ', game_date, "\n",
                                               'Pitch: ', pitch_name, "\n",
                                               'Hit Type: ', events, "\n",
                                               'Exit Velocity (MPH): ', launch_speed, "\n",
                                               'Launch Angle: ', launch_angle, "\n",
                                               'Estimated Distance (ft): ', hit_distance_sc, "\n",
                                               sep = ""))) +
      geom_segment(x = 128, xend = 20, y = -208, yend = -100, size = 0.7, color = "grey66", lineend = "round") +
      geom_segment(x = 128, xend = 236, y = -208, yend = -100, size = 0.7, color = "grey66", lineend = "round") +
      coord_fixed() +
      geom_point(aes(color = events), alpha = 0.6, size = 2) +
      scale_x_continuous(limits = c(25, 225)) +
      scale_y_continuous(limits = c(-225, -25)) +
      labs(title = "",
           subtitle = "") +
      theme_void() +
      theme(plot.margin = margin(0.5, 0.5, 0.5, 0.5, "cm"),
            plot.title = element_text(hjust = 0.5),
            legend.title = element_blank())
  }, width = 400, height = 350)

}

shinyApp(ui = ui, server = server)